# Creating Customer Transaction Dataset
I was faced with a challenge of creating an ecommerce customer transaction dataset using multiple datasets on an eCommerce platform. This company wants to use the new dataset to analyze products and customers to support their marketing campaigns. From this dataset, the business I would create additional datasets containing transaction matrix like order data, customer data, and product data, to enable me perform modeling, analysis, and even reporting of my findings.

I demonstrate the approach using the [Brazilian E-Commerce Public Dataset by Olist on Kaggle](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce). These dataset were selected because like the Olist is an eCommerce platform that connects small business with customers from all over Brazil hence has similar characteristis as the platform I carried out the task for. 

Baring in mind that the project aims to merge all the datasets on the platform into One dataset, to help understand customer activities on the platform, I considered thr following:
- working with a multiple and/or large datasets usually create problems including PC memory issues. Hence, these memory utilization issues must be avoided.
- create dataframes from the core list of dataframes of items purchased which can be used for analysis, modeling, or reporting.
- merge the dataframes into Customer dataframe¶

#### Prerequisites
- jupyter notebook,
- the eCommerce dataset,
- python.

#### The Datasets
The dataset consists of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil. Its features comprises of an order from multiple dimensions: from order status, price, payment and freight performance to customer location, product attributes, reviews written by customers, and geolocation that relates Brazilian zip codes to lat/lng coordinates. 

There is also a marketing funnel dataset from sellers which has information of 8k Marketing Qualified Leads (MQLs) that requested contact between June 1st 2017 and June 1st 2018. These were randomly sampled from the total of MQLs. The features of these datasets allows viewing a sales process from multiple dimensions: lead category, catalog size, behaviour profile, etc.

#### Technology 
Anaconda 


#### Required packages
Numpy, Pandas, Datetime, and Pickle for memory utilization when handling large files without memory utilization issues.

In [1]:
# 1. Import the required packages
import pandas as pd
import numpy as np
import pickle

#### ETL 

We obtain and perform ETL process on the datasets from Kaggle using the following steps. 

1. visit the Kaggle website,

2. select and download all 9 detasets under the [E-Commerce Public Dataset](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce) and the 2 datasets unter the [Marketing Funnel by Olist](https://www.kaggle.com/datasets/olistbr/marketing-funnel-olist),

3. unzip the data which was downloaded as a zip file

4. rename the datasets

5. load the datasets into pandas

6. transform and save dataframes as pickle data

7. dump/delete the origional data

8. load the pickle dataframes for all onward/future usage

In [2]:
# 2. Load the datasets
customers = pd.read_csv('olist_customers_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
productcategory = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
closed_deals = pd.read_csv("olist_closed_deals_dataset.csv")
mql = pd.read_csv("olist_marketing_qualified_leads_dataset.csv")

Working with a large datasets usually create problems including PC memory issues. To get around this, we would pickle each dataset by passing each of the dataframes to a `pd.pickle()` function and then saving them as `_df.pickle` files in the current working directory. This would speed up the storage/retrival and use i.e. merging the datasets into a larger dataset without memory utilization issues.  

In [3]:
# Save the dataframe to a pickle file 
customers.to_pickle('customers_df.pickle')
geolocation.to_pickle('geolocation_df.pickle')
items.to_pickle('items_df.pickle')
orders.to_pickle('orders_df.pickle')
payments.to_pickle('payments_df.pickle')
products.to_pickle('products_df.pickle')
reviews.to_pickle('reviews_df.pickle')
sellers.to_pickle('sellers_df.pickle')
productcategory.to_pickle('productcategory_df.pickle')
closed_deals.to_pickle("closed_deals_df.pickle")
mql.to_pickle("mql_df.pickle")

We now dump/delete the origional dataFrames from the drive.

In [4]:
# delete the original DataFrames
del customers
del geolocation
del items
del orders 
del payments
del products 
del reviews
del sellers
del productcategory 
del closed_deals
del mql

From this point onwards, we would use the pickled dataframes by passing the `pd_df.pickle` dataframes to `pd.read_pickle()` function. This will speedup storage and retrival especially when merging the dataset withought the moemory utilization issues.
### Creating the Customer Transaction Dataframe
We want to create a dataset that can be one surce of data that can be used to create a number of other useful datasets to help analyze the eCommerce customer activities. From the core list we can create additional datasets which can be used for analysis, reporting, or modeling. We can also obtain the standar metrics, such as sum, count, min, max, nunique, standard deviation, as well as passing `numpy functions` and create other eCommerce custom metrics. 

We begin by loading the pickled dataframes for this and other projects using `pd.read_pickle()` function and inspect. 

In [5]:
# reload pickle customers dfs
customers = pd.read_pickle('customers_df.pickle')
# inspect
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


We notice that some fields like `customer_id` and `customer_unique_id` helps identify the customer; `customer_city`, `customer_state` and `customer_zip_code_prefix` indicates the custmer location. Some of these fields are not necessary for the task so we would drop them. 

According to the meta data reveals the following:
- comparatively there are 99,441 `unique customer_id` against 96,096 `customer_unique_id`. As a result, we would drop `customer_unique_id` and keep `customer_id`. 
- the `customer_zip_code_prefix` are five digit codes. However, we notice that there are four and five `customer_zip_code_prefix` is made up of four and five digit codes.

We would also drop the unwanted fields like `customer_city` and `customer_state`, and rename the remaiing fields to reflect their true meaning.

In [6]:
# drop unwanted columns 
customers = customers.drop(columns=["customer_unique_id", "customer_city","customer_state"])

# rename columns
customers.columns = ["customer_id", "zipcode"]

# inspect
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  99441 non-null  object
 1   zipcode      99441 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


We proceed to load and inspect the first few rows of the geolocation dataset.

In [7]:
# load pickle geolocation df
geolocation = pd.read_pickle('geolocation_df.pickle')
# inspect
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


We would rename the columns to make them more meaningful.

In [8]:
# rename columns
geolocation.columns = ["zipcode","latitude","longitude", "city", "state"]

# data info
geolocation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   zipcode    1000163 non-null  int64  
 1   latitude   1000163 non-null  float64
 2   longitude  1000163 non-null  float64
 3   city       1000163 non-null  object 
 4   state      1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


We load the items dataframe and inspect.

In [9]:
# load pickle items df
items = pd.read_pickle('items_df.pickle')
# inspect data head
items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


All feilds for this datafields would be required so we would only rename some of the columns.

In [10]:
# drop unwanted columns
items = items.drop(columns=["seller_id"])
# rename columns
items.columns =['order_id','item_id','product_id','expected_shipping_date','price','delivery_cost']
# inspect data info
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                112650 non-null  object 
 1   item_id                 112650 non-null  int64  
 2   product_id              112650 non-null  object 
 3   expected_shipping_date  112650 non-null  object 
 4   price                   112650 non-null  float64
 5   delivery_cost           112650 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 5.2+ MB


We load and inspect the orders dataframe.

In [11]:
# load pickle orders df 
orders = pd.read_pickle('orders_df.pickle')
# inspect orders data
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


We need all the field so we only rename where necessary.

In [12]:
# rename columns
orders.columns = ['order_id','customer_id','order_status','date_ordered','date_approved','order_with_courier_date',
                  'date_delivered', 'estimated_delivery_date']
# inspect data info
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   order_id                 99441 non-null  object
 1   customer_id              99441 non-null  object
 2   order_status             99441 non-null  object
 3   date_ordered             99441 non-null  object
 4   date_approved            99281 non-null  object
 5   order_with_courier_date  97658 non-null  object
 6   date_delivered           96476 non-null  object
 7   estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


Next we load and inspect the payments dataframe.

In [13]:
# load pickle payments df
payments = pd.read_pickle('payments_df.pickle')
# inspect payments data
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


This is a financial data hence undestanding some of terminologies is required. We have the following:

- payment sequential i.e. a schedule where the payments are made sequentially (one after the other).
- payment value i.e. the amount paied for the order.
- payment value i.e. total amount paid for the order.

We proceed to rename columns where necessary.

In [14]:
# rename columns
payments.columns = ['order_id','payment_sequence','payment_type','payment_installments','order_value']
# inspect data info
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequence      103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   order_value           103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


We load and inspect the products dataframe.

In [15]:
# load pickle products df
products = pd.read_pickle('products_df.pickle')
# inspect products data
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


We drop unwanted columns because they just decribe the product. We then rename the remaining columns. 

In [16]:
# drop unwanted columns
products = products.drop(columns=['product_name_lenght','product_description_lenght',
                                'product_photos_qty','product_weight_g','product_length_cm',
                                'product_height_cm','product_width_cm'])
# rename columns
products.columns = ['product_id','product_category']
# check data info
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        32951 non-null  object
 1   product_category  32341 non-null  object
dtypes: object(2)
memory usage: 515.0+ KB


We load and inspect the reviews dataframe.

In [17]:
# load pickle reviews df
reviews = pd.read_pickle('reviews_df.pickle')
# inspect reviews data
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


We do not need review_creation_date and review_answer_timestamp so we drop them and rename the remaining columns.

In [18]:
# drop unwanted
reviews = reviews.drop(columns=['review_creation_date','review_answer_timestamp'])
# rename columns
reviews.columns = ['review_id','order_id','review_score','review_headline','review_body']
# check info
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   review_id        99224 non-null  object
 1   order_id         99224 non-null  object
 2   review_score     99224 non-null  int64 
 3   review_headline  11568 non-null  object
 4   review_body      40977 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


We load and inspect the sellers dataframe.

In [19]:
# load pickle sellers df
sellers = pd.read_pickle('sellers_df.pickle')
# inspect sellers data
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [20]:
# drop unwanted columns
sellers = sellers.drop(columns=["seller_city","seller_state"])
# rename columns
sellers.columns = ['seller_id','zipcode']
# inspect data info
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   seller_id  3095 non-null   object
 1   zipcode    3095 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 48.5+ KB


We load and inspect the product category dataframe.

In [21]:
# load pickle productcategory df
productcategory = pd.read_pickle('productcategory_df.pickle')
# inspect product category
productcategory.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


We drop product_category_name and rename product_category_name_english.

In [22]:
# drop unwanted column
productcategory = productcategory.drop(columns=['product_category_name'])
# rename column
productcategory.columns = ['product_category']
# inspect data info
productcategory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_category  71 non-null     object
dtypes: object(1)
memory usage: 696.0+ bytes


We now repeate the process on the marketing funnel datasets. For clarity, the meta data when a qualified lead fills in a form at a landing page of the website,he is contacted by a sales development representative. At this step some information is checked and more information about the lead is gathered. So we proceed to laod and inspect the closed deals dataframe.

In [23]:
# load pickle closed_deal df
closed_deals = pd.read_pickle('closed_deals_df.pickle')
# inspect closed_deal category
closed_deals.head()

,mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,5420aad7fec3549a85876ba1c529bd84,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,NaN,NaN,NaN,reseller,NaN,0.0
1,a555fb36b9368110ede0f043dfc3b9a0,bbb7d7893a450660432ea6652310ebb7,09285259593c61296eef10c734121d5b,d3d1e91a157ea7f90548eef82f1955e3,2018-05-08 20:17:59,car_accessories,industry,eagle,NaN,NaN,NaN,reseller,NaN,0.0
2,327174d3648a2d047e8940d7d15204ca,612170e34b97004b3ba37eae81836b4c,b90f87164b5f8c2cfa5c8572834dbe3f,6565aa9ce3178a5caf6171827af3a9ba,2018-06-05 17:27:23,home_appliances,online_big,cat,NaN,NaN,NaN,reseller,NaN,0.0
3,f5fee8f7da74f4887f5bcae2bafb6dd6,21e1781e36faf92725dde4730a88ca0f,56bf83c4bb35763a51c2baab501b4c67,d3d1e91a157ea7f90548eef82f1955e3,2018-01-17 13:51:03,food_drink,online_small,NaN,NaN,NaN,NaN,reseller,NaN,0.0
4,ffe640179b554e295c167a2f6be528e0,ed8cb7b190ceb6067227478e48cf8dde,4b339f9567d060bcea4f5136b9f5949e,d3d1e91a157ea7f90548eef82f1955e3,2018-07-03 20:17:45,home_appliances,industry,wolf,NaN,NaN,NaN,manufacturer,NaN,0.0


Here our fields of interest are mql_id and seller_id so we drop the unwanted fields. 

In [24]:
# drop unwanted column
closed_deals = closed_deals.drop(columns=['sdr_id','sr_id','business_segment','lead_type','lead_behaviour_profile',
                                               'has_company','has_gtin','average_stock','declared_product_catalog_size',
                                               'declared_monthly_revenue'])
# rename column
closed_deals.columns = ['mql_id','seller_id','won_date','business_type']
# inspect data info
closed_deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   mql_id         842 non-null    object
 1   seller_id      842 non-null    object
 2   won_date       842 non-null    object
 3   business_type  832 non-null    object
dtypes: object(4)
memory usage: 26.4+ KB


Finally, we load and inspect the mql dataframe.

In [25]:
# load pickle closed_deal df
mql = pd.read_pickle('mql_df.pickle')
# inspect closed_deal category
mql.head()

,mql_id,first_contact_date,landing_page_id,origin
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search


In [26]:
# rename column
mql.columns = ['mql_id','first_contact_date','landing_page','page_origin']
# inspect data info
mql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mql_id              8000 non-null   object
 1   first_contact_date  8000 non-null   object
 2   landing_page        8000 non-null   object
 3   page_origin         7940 non-null   object
dtypes: object(4)
memory usage: 250.1+ KB


#### Merge DataFrames into Customer Dataframe
We pass the multiple dataframes to a user defined function called `preprocess_dfs()` to merge all 11 dataframes together to form the customer dataframe. We pass the `second dataframe`, how the merge should occur, and pass the key to the `pd.merge(second pd, how = 'side', on= 'key')` to show how the merge should be made. We apply the following approach:
1. We select the entire customers dataframe, position it on the left and join (`left` join) it with the geolocation dataframe, using the zipcode (i.e. on =`zipcode`) key
2. We seletect the entire output (i.e. new dataframe obtained) and join it with the sellers dataframe using the `zipcode` key. But because the `zipcode` field in the output is inner, we apply `inner` join by passing `inner` to how (i.e. how=`inner`).
3. We merge the output and the orders dataframe by placing the entire output on the right (`right` join) of the orders dataframe on `customer_id`. 
4. We merge the new output with the payments dataframe on `order_id` using `inner` join.
5. We merge the new output with the items dataframe on `order_id` using `inner` join.
6. We merge the new output with the reviews dataframe on `order_id` using `inner` join.
7. We merge the output with the products dataframe on `product_id` using `inner` join.
8. We merge the output with the closed_deals dataframe on the `seller_id` using `inner` join.
9. We merge the output with the mql dataframe on the `mql_id` using `inner` join.
10. Finally we merge the output with the productcategory dataframe on `product_category` using `inner` join.

In [27]:
# define function to merge all dataframes
def preprocess_dfs():
    """
    Merge multiple dfs
    """
    # customers & geolocation dfs, left join, on zipcode
    output = customers.merge(geolocation, how='left', on= 'zipcode')
    output = output.merge(sellers, how='inner', on= 'zipcode')
    output = output.merge(orders, how='right',on='customer_id')
    output = output.merge(payments, how='inner',on='order_id')
    output = output.merge(items, how='inner',on='order_id')
    output = output.merge(reviews, how='inner',on='order_id')
    output = output.merge(products, how='inner',on='product_id')
    output = output.merge(closed_deals, how='inner',on='seller_id')
    output = output.merge(mql, how='inner',on='mql_id')
    output = output.merge(productcategory, how='inner',on='product_category')    
    
    return output

We call the function to create the customer dataframe. We proceed to inspect the dataframe attributes.

In [28]:
# call function to get DataFrame
df_cust = preprocess_dfs()
# inspect
df_cust.head(10).T

,0,1,2,3,4,5,6,7,8,9
customer_id,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b
zipcode,88032.0,88032.0,88032.0,88032.0,88032.0,88032.0,88032.0,88032.0,88032.0,88032.0
latitude,-27.536913,-27.470734,-27.556345,-27.557115,-27.553879,-27.551317,-27.540764,-27.551561,-27.470734,-27.558472
longitude,-48.509018,-48.468516,-48.493552,-48.496774,-48.500481,-48.496324,-48.505618,-48.493554,-48.468516,-48.498091
city,florianopolis,florianopolis,florianopolis,florianopolis,florianópolis,florianopolis,florianopolis,florianopolis,florianopolis,florianopolis
state,SC,SC,SC,SC,SC,SC,SC,SC,SC,SC
seller_id,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d
order_id,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a
order_status,delivered,delivered,delivered,delivered,delivered,delivered,delivered,delivered,delivered,delivered
date_ordered,2017-05-11 23:24:53,2017-05-11 23:24:53,2017-05-11 23:24:53,2017-05-11 23:24:53,2017-05-11 23:24:53,2017-05-11 23:24:53,2017-05-11 23:24:53,2017-05-11 23:24:53,2017-05-11 23:24:53,2017-05-11 23:24:53


In [29]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91472 entries, 0 to 91471
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              91472 non-null  object 
 1   zipcode                  91472 non-null  float64
 2   latitude                 91472 non-null  float64
 3   longitude                91472 non-null  float64
 4   city                     91472 non-null  object 
 5   state                    91472 non-null  object 
 6   seller_id                91472 non-null  object 
 7   order_id                 91472 non-null  object 
 8   order_status             91472 non-null  object 
 9   date_ordered             91472 non-null  object 
 10  date_approved            91472 non-null  object 
 11  order_with_courier_date  90423 non-null  object 
 12  date_delivered           90159 non-null  object 
 13  estimated_delivery_date  91472 non-null  object 
 14  payment_sequence      

We notice that the new dataframe contains 91,472 entries with 34 fields. Interestingly, about 24.4 MB of the memory was used. We proceed to save the new dataframe as a pickle file to speed up the storage/retrival and use i.e. merging the datasets into a larger dataset without memory utilization issues. After this, we dump/delete the origional dataFrame from the drive. We would now use new pickle dataframe for all onward analysis.

In [30]:
# Save the dataframe to a pickle file 
df_cust.to_pickle('df_cust_df.pickle')

In [31]:
# dump df
del df_cust

#### Inspecting the Customer Transaction Dataset
We can load the dataframe as our custome transaction dataset and inspect.

In [32]:
# reload new df
cust_transactions = pd.read_pickle("df_cust_df.pickle")
# inspect
cust_transactions.head(5).T

,0,1,2,3,4
customer_id,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b,d5695e900ecca93e1369d850dc1e986b
zipcode,88032.0,88032.0,88032.0,88032.0,88032.0
latitude,-27.536913,-27.470734,-27.556345,-27.557115,-27.553879
longitude,-48.509018,-48.468516,-48.493552,-48.496774,-48.500481
city,florianopolis,florianopolis,florianopolis,florianopolis,florianópolis
state,SC,SC,SC,SC,SC
seller_id,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d,c0aff2da32c17759d30b22fb0af6649d
order_id,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a,5c1ae73856bf1d09958ec5ae3a64b73a
order_status,delivered,delivered,delivered,delivered,delivered
date_ordered,2017-05-11 23:24:53,2017-05-11 23:24:53,2017-05-11 23:24:53,2017-05-11 23:24:53,2017-05-11 23:24:53


We can now use the dataframe for onward activities to support the eCommerce campaigns.

#### George Jordan, Tuesday April 11 2023.